### Preparación de las imágenes 

BIANCA trabaja en el espacio del sujeto, sin embargo todas las modalidades MRI requieren ser registradas a una imagen base en común y tener las mismas dimensiones en términos de resolución y FOV. 

1. Selecciona la imagen base. Esta será el espacio de referencia para las imágenes de entrada y salida. 

2. Ejecute extracción de cerebro (BET) en al menos una modalidad con el fin de obtener una máscara cerebral. Si desea restringir aún más el área donde se detectarán las lesiones (y reducir los falsos positivos), puede considerar el enmascaramiento previo como se describe a continuación: 

    Si ve hiperintensidades de falsos positivos debido a artefactos como los de pulsación de LCR en FLAIR, podría ser útil aplicar una máscara para excluir las regiones afectadas. Tenga en cuenta que BIANCA no está optimizado para la segmentación de lesiones yuxtacorticales, cerebelosas y subcorticales.

    El siguiente script crea un ejemplo de máscara de inclusión de imágenes T1, que excluye sustancia gris cortical y las siguientes estructuras: putamen, globo pálido, núcleo accumbens, tálamo, tronco encefálico, cerebelo, hipocampo, amígdala. la sustancia gris cortical se excluye de la máscara cerebral extrayendo el líquido cefalorraquídeo (CSF) cortical del mapa de estimación de volumen parcial (PVE) del CSF de un solo sujeto, dilatándolo para alcanzar la sustancia gris cortical y excluyendo estas áreas. Las otras estructuras se identifican en el espacio MNI, se registran de forma no lineal en las imágenes de los sujetos individuales y se eliminan de la máscara cerebral.
    
        make_bianca_mask <structural_image> <CSF pve> <warp_file_MNI2structural>                       <keep_intermediate_files>
        
        
    <structural_image>: Nombre de la imagen estructural por ejemplo T1_biascorr. El script trabaja asumiendo que la imagen BET se llamará <structural image>_brain.nii.gz.
    <CSF pve>: mapa de de estimación de volumen parcial de CSF, por ejemplo la salida de FSL-FAST
    <warp_file_MNI2structural>: corresponde al archivo transformación no lineal del espacio estándar a la imagen estructural. Si se ejecuta fsl_anat, es posible usar el archivo MNI_to_T1_nonlin_field.nii.gz en el directorio de salida. Si se tiene el archivo de transformación de la imagen estructural al espacio estándar MNI, es posible calcular la inversa con el comando invwarp -w warpvol -o invwarpvol -r refvol. 
    <keep_intermediate_files>: al indicar 1 en este argumento no se eliminará la carpeta con archivos temporales.

3. Registre todas las modalidades a la imagen base para cada sujeto

#### Preparación del dataset de entrenamiento

El algoritmo requiere un conjunto de entresamiento con voxeles pre-clasificados, por ejemplo imagenes manualmente segmentadas. Estas son usadas para crear un conjunto de vectores característicos para las clases *lesión* y *no lesión*

Las mascaras de lesión que será utilizadas como conjunto de entrenamiento deben cumplir los siguientes requisitos:

- Binaria (1=lesion; 0=no-lesion)
- Formato nifti (nii.gz) 
- En el mismo espacio de la imagen base. Si la segmentación manual fue realizada sobre una imagen en un espacio diferente al espacio base, la mascara de lesión deberá registrarse y transofmrse al espacio de la imagen base. 

#### Ejecutar BIANCA
BIANCA requiere un archivo maestro que contiene una fila por sujeto (por ejemplo T1 y FLAIR) y en cada fila una lista de todos los archivos necesarios para ese sujeto:

Las imágenes que se desean usar para clasificación (por ejemplo T1 y FLAIR),todas corregistradas al espacio base. 
1. Una imagen correspondiente al cerebro extraido con el fin de obtejer una máscara del cerebro.
2. Una máscara binaria de la lesión obtenida manualmente corregistrada al espacio base.
3. Matriz de transfornmación del espacio del sujeto al espacio estándar. Necesaria para calcular características espaciales a partir de coordenadas MNI. 

Es decir: 

FLAIR_brain.nii.gz : imagen estructural principal, con extracción de cerebro
FLAIR_Lesion_mask.nii.gz : máscara manual de lesión del sujeto, binaria para ser usada en el entrenamiento de BIANCA 
FLAIR_brain_to-MNI_xfm.mat : matriz de transformación del espacio del sujeto (imagen estructural principal) al espacio estándar (opcional). Este archivo permitirá el uso de características espaciales (coordenadas MNI)
T1_brain_to-FLAIR_brain.nii.gz : entrada adicional (opcional). Otras modalidades que puedan ayudar en la segmentación de la lesión (por ejemplo la imagen T1), todas registradas a la imagen principal.La imagen T1 a utilizar se obtiene de la siguiente manera: 

![](captura_bianca1a.png)

### Datos ejemplo
##### Descargue los datos disponibles en: https://fsl.fmrib.ox.ac.uk/fslcourse/  'Structural data for practical 2'
![](captura_bianca1.png)

##### Al descomprimir observará los siguientes archivos: 

![](captura_bianca2.png)

#####  Acceda a la carpeta bianca para iniciar el trabajo con los datos 

![](captura_bianca3.png)


#### El archivo masterfile.txt contiene la siguiente información: 
![](captura_bianca4.png)

bianca --singlefile=masterfile.txt --labelfeaturenum=4 --brainmaskfeaturenum=1 --querysubjectnum=1 --trainingnums=1,2,3,4,5,6,7,8,9,10 --featuresubset=1,2 --matfeaturenum=3 --trainingpts=2000 --nonlespts=10000 --selectpts=noborder -o sub001_bianca_output –v

#### --labelfeaturenum=4: 
 Con este comando BIANCA usará datos del archivo masterfile.txt. Buscará información sobre las imagenes pre-etiquetadas en la cuarta columna. 

#### --querysubjectnum=1
 si --querysubjectnum está incluido en la lista de --trainingnums, este será automáticamente excluido del conjunto de entrenamiento. De esta manera podemos observar qué tan bien la máscara de lesión obtenida con BIANCA se compara con la máscara manual

#### --brainmaskfeaturenum=1
El sujeto a segmentar es el especificado en la primera columna del archivo masterfile.txt. Dado que este sujeto también es uno de los sujetos de entrenamiento, BIANCA utilizará solo los 9 restantes para el entrenamiento (como el enfoque LOO, para evitar sesgos y sobreajuste) 

#### --featuresubset=1,2 
BIANCA utilizará como características espaciales las imágenes en la primera y segunda columnas del archivo maestro. Este parámetro contiene una Lista de números de columnas (separados por comas, sin espacios) asociadas al archivo maestro que contiene el nombre de las imágenes para usar como características de intensidad (1,2 en el ejemplo anterior para usar FLAIR [sub-001/FLAIR_brain.nii.gz] y T1 [sub-001/T1_brain_to-FLAIR_brain.nii.gz]) (predeterminado: usar todas las modalidades como características). La imagen utilizada para especificar la máscara distinta de cero (cerebro) debe ser parte del subconjunto de características.


#### --matfeaturenum=3
número de columna en el archivo maestro de archivos de matriz (matriz de transformación del espacio del sujeto al espacio MNI). Necesario para extraer características espaciales (coordenadas MNI; 3 en el ejemplo anterior)

For the training, BIANCA will use, for each training subject, (up to) 2000 points among the voxels labeled as lesion and (up to) 10000 points among the non-lesion voxels, excluding voxels close to the lesion’s edge.

#### -o sub001_bianca_output –v
La imagen de salida tendrá por nombre: sub001_bianca_output. Verbose mode is on.

In [3]:
%%bash
cd /Users/neurofuncional/Downloads/check_bianca_2nd/seg_struc/bianca
bianca --singlefile=masterfile.txt --trainingnums=1,2,3,4,5,6,7,8,9 --labelfeaturenum=4 --querysubjectnum=10 --brainmaskfeaturenum=1 --featuresubset=1,2 --matfeaturenum=3 --trainingpts=2000 --nonlespts=10000 --selectpts=noborder -o bianca_output

